In [92]:

import os
from datetime import datetime
import requests
import time
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')
from collections import Counter

In [16]:
f'https://api.nytimes.com/svc/search/v2/articlesearch.json?&sort=newest&q=oil,naturalgas,energy,crude&api-key=Bmf4GKCtrsMvIyxf7DFl8HefYqv3cZK5'

'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=oil,naturalgas&api-key=Bmf4GKCtrsMvIyxf7DFl8HefYqv3cZK5'

In [10]:
def call_nyt_by_year_and_page(year, page):
    root_url   = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=oil,naturalgas,energy,crude&sort=newest'
    begin_date = '&begin_date={}0101'.format(str(year))
    end_date   = '&end_date={}1231'.format(str(year))
    pagination = '&page={}'.format(str(page))
    doc_params = '&fl=pub_date&fl=abstract'
    
    url        = root_url + begin_date + end_date + pagination + doc_params + '&api-key=Bmf4GKCtrsMvIyxf7DFl8HefYqv3cZK5'
    
    
    response   = requests.get(url)
    
    url_data = response.json()
    
    return url_data

In [11]:
# request all pages of articles for a single year
# returns a list of article dictionaries
def call_nyt_by_year(year):
    annual_articles = []
    
    # find how many articles (hits) contain 'Oil& related keyword' in a calendar year
    total_results = call_nyt_by_year_and_page(year,0)
    hits  = total_results['response']['meta']['hits']
    
    # request all available pages
    # sleep 7 seconds to clear NYT API rate limit
    for i in range(int(hits/10)):
        query = call_nyt_by_year_and_page(year,i)
        annual_articles = annual_articles + query['response']['docs']
        time.sleep(7)
        
    return annual_articles

In [12]:
# request all pages of articles for all years
# returns a list of article dictionaries
def call_nyt_by_all_years(years):
    all_articles = []
    
    for year in years:
        annual_articles = call_nyt_by_year(year)
        all_articles = all_articles + annual_articles
        time.sleep(7)
        
    return all_articles

In [13]:

# create a list of dictionaries of all NYT articles containing oil or oilrelated data from 2010 to 2019
all_years    = list(range(2010,2020))
all_articles = call_nyt_by_all_years(all_years)

In [14]:
# all_articles[0]["abstract"]
text_df= pd.DataFrame({
    'text':[x['abstract'] for x in all_articles]
})
text_df

,text
0,"It was a year full of big environmental news, ..."
1,About $11 billion in oil and natural gas trade...
2,"Recent years have been humbling for oil bulls,..."
3,Crude oil traded over $90 for a fifth consecut...
4,Over the last three presidential administratio...
...,...
605,"Chevron, the No. 2 energy producer in the Unit..."
606,The president wants to limit the influence of ...
607,"Chevron, Occidental Petroleum and BHP have inv..."
608,My mother-in-law wouldn’t tell us what our wed...


In [15]:
text_df['text']

0      It was a year full of big environmental news, ...
1      About $11 billion in oil and natural gas trade...
2      Recent years have been humbling for oil bulls,...
3      Crude oil traded over $90 for a fifth consecut...
4      Over the last three presidential administratio...
                             ...                        
605    Chevron, the No. 2 energy producer in the Unit...
606    The president wants to limit the influence of ...
607    Chevron, Occidental Petroleum and BHP have inv...
608    My mother-in-law wouldn’t tell us what our wed...
609    Officials had told residents to stay indoors f...
Name: text, Length: 610, dtype: object

In [35]:
master_df = pd.DataFrame(all_articles)
master_df

,abstract,pub_date
0,"It was a year full of big environmental news, ...",2010-12-31T23:00:27+0000
1,About $11 billion in oil and natural gas trade...,2010-12-30T20:21:30+0000
2,"Recent years have been humbling for oil bulls,...",2010-12-30T02:34:46+0000
3,Crude oil traded over $90 for a fifth consecut...,2010-12-29T14:06:12+0000
4,Over the last three presidential administratio...,2010-12-24T05:49:08+0000
...,...,...
605,"Chevron, the No. 2 energy producer in the Unit...",2019-04-12T11:08:22+0000
606,The president wants to limit the influence of ...,2019-04-10T21:40:17+0000
607,"Chevron, Occidental Petroleum and BHP have inv...",2019-04-07T18:52:03+0000
608,My mother-in-law wouldn’t tell us what our wed...,2019-03-22T01:38:25+0000


In [37]:
master_df['pub_date'] = pd.to_datetime(master_df['pub_date'])

In [33]:
datetime.now()

datetime.datetime(2020, 2, 8, 10, 48, 1, 332777)

In [39]:
master_df['pub_date'] = master_df['pub_date'].dt.strftime('%Y-%m-%d')

In [40]:
master_df

,abstract,pub_date
0,"It was a year full of big environmental news, ...",2010-12-31
1,About $11 billion in oil and natural gas trade...,2010-12-30
2,"Recent years have been humbling for oil bulls,...",2010-12-30
3,Crude oil traded over $90 for a fifth consecut...,2010-12-29
4,Over the last three presidential administratio...,2010-12-24
...,...,...
605,"Chevron, the No. 2 energy producer in the Unit...",2019-04-12
606,The president wants to limit the influence of ...,2019-04-10
607,"Chevron, Occidental Petroleum and BHP have inv...",2019-04-07
608,My mother-in-law wouldn’t tell us what our wed...,2019-03-22


In [43]:
master_df.sort_values(by=['pub_date'], inplace=True, ascending=True)

In [44]:
master_df

,abstract,pub_date
79,The global recession had a greater impact than...,2010-01-01
78,The two countries failed to renew an agreement...,2010-01-04
77,"The quarrel, over an oil subsidy that has help...",2010-01-05
76,"Traders seem hesitant about several sectors, p...",2010-01-07
74,A member of the United States-China Economic a...,2010-01-08
...,...,...
584,Teenage comments in response to our recent wri...,2019-11-21
583,It remains to be seen whether the reduction in...,2019-12-05
581,Chevron’s multibillion-dollar write-down of ga...,2019-12-11
582,The oil giant’s shares reached prices on the R...,2019-12-11


In [50]:
all_news_per_day = []
#Iterating through unique dates
for day in master_df['pub_date'].unique():
    #Finding all records with the corresponding day
    corr_day = master_df.loc[master_df['pub_date']== day]
    # Combine the news for those days
    combined_news = " ".join(s for s in corr_day["abstract"])
    #appending combined news to all news per day with the current day
    all_news_per_day.append((combined_news,day))

In [54]:
combined_news_day = pd.DataFrame(all_news_per_day,columns=['combined_absract', 'Date'])

In [55]:
combined_news_day

,combined_absract,Date
0,The global recession had a greater impact than...,2010-01-01
1,The two countries failed to renew an agreement...,2010-01-04
2,"The quarrel, over an oil subsidy that has help...",2010-01-05
3,"Traders seem hesitant about several sectors, p...",2010-01-07
4,A member of the United States-China Economic a...,2010-01-08
...,...,...
522,The timing of the announcement suggested an ur...,2019-11-15
523,Teenage comments in response to our recent wri...,2019-11-21
524,It remains to be seen whether the reduction in...,2019-12-05
525,Chevron’s multibillion-dollar write-down of ga...,2019-12-11


In [74]:
tokens = []
lemma = []

for doc in nlp.pipe(combined_news_day['combined_absract'].astype('unicode').values, batch_size=50,
                            n_threads=3):
    if doc.is_parsed:
            tokens.append([n.text for n in doc])
            lemma.append([n.lemma_ for n in doc])

    else:
            # We want to make sure that the lists of parsed results have the
            # same number of entries of the original Dataframe, so add some blanks in case the parse fails
            tokens.append(None)
            lemma.append(None)

    combined_news_day['abstract_tokens'] = tokens
    combined_news_day['abstract_lemma'] = lemma

ValueError: Length of values does not match length of index

In [72]:

combined_news_day['new_col'] = combined_news_day['combined_absract'].apply(lambda x: nlp(x))

In [73]:
combined_news_day

,combined_absract,Date,new_col
0,The global recession had a greater impact than...,2010-01-01,"(The, global, recession, had, a, greater, impa..."
1,The two countries failed to renew an agreement...,2010-01-04,"(The, two, countries, failed, to, renew, an, a..."
2,"The quarrel, over an oil subsidy that has help...",2010-01-05,"(The, quarrel, ,, over, an, oil, subsidy, that..."
3,"Traders seem hesitant about several sectors, p...",2010-01-07,"(Traders, seem, hesitant, about, several, sect..."
4,A member of the United States-China Economic a...,2010-01-08,"(A, member, of, the, United, States, -, China,..."
...,...,...,...
522,The timing of the announcement suggested an ur...,2019-11-15,"(The, timing, of, the, announcement, suggested..."
523,Teenage comments in response to our recent wri...,2019-11-21,"(Teenage, comments, in, response, to, our, rec..."
524,It remains to be seen whether the reduction in...,2019-12-05,"(It, remains, to, be, seen, whether, the, redu..."
525,Chevron’s multibillion-dollar write-down of ga...,2019-12-11,"(Chevron, ’s, multibillion, -, dollar, write, ..."


In [78]:
combined_news_day

527

In [89]:
tokens = []
lemma = []
for doc in nlp.pipe(combined_news_day['combined_absract'].astype('unicode').values, batch_size=50,
                        n_threads=3):
    if doc.is_parsed:
        tokens.append([n.text for n in doc])
        lemma.append([n.lemma_ for n in doc
                        if not n.is_stop
                        and not n.is_punct])
    else:
        # We want to make sure that the lists of parsed results have the
        # same number of entries of the original Dataframe, so add some blanks in case the parse fails
        tokens.append(None)
        lemma.append(None)
combined_news_day['abstract_tokens'] = tokens
combined_news_day['abstract_lemma'] = lemma

In [91]:
combined_news_day.drop(['new_col'],axis=1)

,combined_absract,Date,abstract_tokens,abstract_lemma
0,The global recession had a greater impact than...,2010-01-01,"[The, global, recession, had, a, greater, impa...","[global, recession, great, impact, diplomatic,..."
1,The two countries failed to renew an agreement...,2010-01-04,"[The, two, countries, failed, to, renew, an, a...","[country, fail, renew, agreement, crude, oil, ..."
2,"The quarrel, over an oil subsidy that has help...",2010-01-05,"[The, quarrel, ,, over, an, oil, subsidy, that...","[quarrel, oil, subsidy, help, prop, Belarus, l..."
3,"Traders seem hesitant about several sectors, p...",2010-01-07,"[Traders, seem, hesitant, about, several, sect...","[trader, hesitant, sector, particularly, housi..."
4,A member of the United States-China Economic a...,2010-01-08,"[A, member, of, the, United, States, -, China,...","[member, United, States, China, Economic, Secu..."
...,...,...,...,...
522,The timing of the announcement suggested an ur...,2019-11-15,"[The, timing, of, the, announcement, suggested...","[timing, announcement, suggest, urgent, scramb..."
523,Teenage comments in response to our recent wri...,2019-11-21,"[Teenage, comments, in, response, to, our, rec...","[teenage, comment, response, recent, writing, ..."
524,It remains to be seen whether the reduction in...,2019-12-05,"[It, remains, to, be, seen, whether, the, redu...","[remain, see, reduction, output, impact, price..."
525,Chevron’s multibillion-dollar write-down of ga...,2019-12-11,"[Chevron, ’s, multibillion, -, dollar, write, ...","[Chevron, multibillion, dollar, write, gas, as..."


In [97]:
counts = []
for x in combined_news_day['abstract_lemma']:
    counts.append(Counter(x).most_common(10))

In [98]:
combined_news_day['counts'] = counts

In [103]:
combined_news_day.loc[combined_news_day['top_word'] == 'gas']

,combined_absract,Date,new_col,abstract_tokens,abstract_lemma,counts,top_word
150,A report found that recharging an electric veh...,2012-04-13,"(A, report, found, that, recharging, an, elect...","[A, report, found, that, recharging, an, elect...","[report, find, recharge, electric, vehicle, pa...","[(gas, 2), (report, 1), (find, 1), (recharge, ...",gas
177,Low prices for consumers. Big profits for bank...,2012-10-20,"(Low, prices, for, consumers, ., Big, profits,...","[Low, prices, for, consumers, ., Big, profits,...","[low, price, consumer, big, profit, banker, ga...","[(gas, 2), (low, 1), (price, 1), (consumer, 1)...",gas
249,Islamist rebels and extremists are using oil a...,2014-01-29,"(Islamist, rebels, and, extremists, are, using...","[Islamist, rebels, and, extremists, are, using...","[islamist, rebel, extremist, oil, gas, proceed...","[(gas, 2), (say, 2), (islamist, 1), (rebel, 1)...",gas
313,Local vintners are fighting a project under wh...,2014-12-25,"(Local, vintners, are, fighting, a, project, u...","[Local, vintners, are, fighting, a, project, u...","[local, vintner, fight, project, ten, million,...","[(gas, 2), (local, 1), (vintner, 1), (fight, 1...",gas
525,Chevron’s multibillion-dollar write-down of ga...,2019-12-11,"(Chevron, ’s, multibillion, -, dollar, write, ...","[Chevron, ’s, multibillion, -, dollar, write, ...","[Chevron, multibillion, dollar, write, gas, as...","[(gas, 2), (Chevron, 1), (multibillion, 1), (d...",gas


In [104]:
combined_news_day['top_word'] = combined_news_day['counts'].apply(lambda x: x[0][0])

In [105]:
combined_news_day['top_word']

0       global
1      country
2      quarrel
3       trader
4        China
        ...   
522     timing
523    teenage
524     remain
525        gas
526       size
Name: top_word, Length: 527, dtype: object

In [1]:
nlp_result=[]
for i in range(combined_news_day.shape[0]):
    nlp_doc=nlp(combined_news_day.iloc[i,0])
    for token in nlp_doc:
        #print(token, token.lemma_)

SyntaxError: unexpected EOF while parsing (<ipython-input-1-3116547a5541>, line 5)

In [86]:
doc = nlp_result
for token in doc:
    print (token, token.lemma)

AttributeError: 'spacy.tokens.doc.Doc' object has no attribute 'lemma'

In [28]:
!pip install requests

In [1]:
pip install spacy

     |████████████████████████████████| 14.2MB 14.9MB/s eta 0:00:01   |████▏                           | 1.8MB 5.9MB/s eta 0:00:03     |████████▊                       | 3.9MB 5.9MB/s eta 0:00:02     |█████████████████               | 7.5MB 5.9MB/s eta 0:00:02
     |████████████████████████████████| 215kB 16.8MB/s eta 0:00:01
     |████████████████████████████████| 276kB 16.0MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 28.3MB/s eta 0:00:01
     |████████████████████████████████| 4.0MB 15.2MB/s eta 0:00:01
     |████████████████████████████████| 61kB 8.8MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [33]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
